<a href="https://colab.research.google.com/github/Mancupfire/Retrieval_Model/blob/main/Retrieval_Model(sBERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric
!pip install transformers

In [ ]:
import json
import numpy as np
import torch
from torch_geometric.data import Data
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Đọc dữ liệu huấn luyện
with open('edinburgh-keywords_train.json', 'r') as f:
    train_data = json.load(f)

keywords = list(train_data['np2count'].keys())

In [ ]:
# Loại bỏ những từ bị trùng
keyword_set = set(keywords)

In [ ]:
def extract_users(info):
    l_user, user2kw = [], []
    for ii in info:
        lus = info[ii]
        for u in lus:
            if u not in l_user:
                l_user.append(u)
                user2kw.append([])
            idx = l_user.index(u)
            user2kw[idx].append(ii)
    return l_user, user2kw

In [ ]:
train_users, train_users2kw = extract_users(train_data['np2users'])

In [ ]:
# restaurant_set = set()
# listres = []
# for kw in train_data['np2rests'].keys():
#     listres.extend(train_data['np2rests'][kw].keys())
# restaurant_set = set(listres)

In [ ]:
# Tạo ma trận liên kết từ keywords và restaurant
keyword_set = list(keyword_set)
restaurant_set = list(restaurant_set)
restaurants = len(listres)
num_keywords = len(keyword_set)
num_restaurants = len(restaurant_set)
a = np.zeros((num_keywords, num_restaurants))

for kw in train_data['np2rests'].keys():
    for res in train_data['np2rests'][kw].keys():
        idx_kw = keyword_set.index(kw)
        idx_res = restaurant_set.index(res)
        a[idx_kw][idx_res] = 1

In [ ]:
# Mã hóa các từ khóa trong tập huấn luyện
keyword_embeddings = model.encode(list(keyword_set))

In [ ]:
# Đọc dữ liệu kiểm tra
with open('edinburgh-keywords_test.json', 'r') as r:
    test_data = json.load(r)

user_keywords = list(test_data['np2reviews'].keys())
user_keywords_list = list(user_keywords)

In [ ]:
test_users, test_users2kw = extract_users(test_data['np2users'])

In [ ]:
# Mã hóa các từ khóa trong tập kiểm tra
test_keywords = [kw for sublist in test_users2kw for kw in sublist]
test_keyword_embeddings = model.encode(test_keywords)

In [ ]:
# Tính toán độ tương đồng giữa từ khóa kiểm tra và từ khóa huấn luyện (Em sử dụng công thức consine similarity)
similarity_scores = cosine_similarity(test_keyword_embeddings, keyword_embeddings)

In [ ]:
# Thiết lập ngưỡng độ tương đồng (Cái này em đang set chỉ giữ lại các từ có độ tương đồng trên 0.6)
threshold = 0.6
filtered_keywords = []

for i, test_kw in enumerate(test_keywords):
    if np.max(similarity_scores[i]) > threshold:
        filtered_keywords.append(test_kw)

print("Filtered Keywords:", filtered_keywords)

Filtered Keywords: ['flatmates', 'food', 'while', 'menu', 'door', 'fresher week', 'hall', 'usual range', 'chinese classic dishes', 'crispy chilli shredded beef', 'fried rice', 'bag', 'prawn crackers', 'minutes', 'reality', 'fiver', 'beef', 'good quality', 'spicy sauce', 'bit', 'wait', 'friend', 'meal', 'price', 'standard', 'quality', 'shop', 'chance', 'hands', 'rice', 'flavours', 'decor', 'restaurant', 'part', 'soup', 'size', 'way', 'dishes', 'taste', 'edinburgh', 'tea', 'place', 'weekend', 'table', 'day', 'need', 'guy', 'sign', 'lot', 'head', 'selection', 'group', 'tomato', 'chicken', 'pastries', 'main course', 'dad', 'eye', 'service', 'fish', 'option', 'variety', 'sauce', 'fish bar', 'pacific', 'name', 'great food', 'surprising prices', 'friends house party', 'speed', 'general atmosphere', 'cool fish theme', 'style', 'chips', 'haddock', 'counter', 'hour', 'fluffy chips', 'can', 'coke', 'portion', 'pacific fish bar', 'great easy cheap dinner', 'life', 'experience', 'fact', 'dumplings'

In [ ]:
# Cập nhật lại danh sách test_users2kw sau khi loại bỏ những từ khóa không có nghĩa
for i, user_kw in enumerate(test_users2kw):
    test_users2kw[i] = [kw for kw in user_kw if kw in filtered_keywords]

In [ ]:
keyword_set = list(keyword_set)

In [ ]:
# Xử lý lại với các từ khóa đã lọc

for kw in test_users2kw:
    t = np.zeros((1, len(keyword_set)))
    keywords = kw[:10]
    for keys in keywords:
        if keys in keyword_set:
            idx_kw = keyword_set.index(keys)
            t[0][idx_kw] = 1
    R = np.dot(t, a)
    result = np.argsort(R[0])[::-1][:10]

In [ ]:
result

array([646, 227, 679, 918, 605, 420, 346, 604, 451, 929])